# Models Work

In [9]:
! rm -r project_chd/

In [10]:
! git clone https://github.com/JulianKrese/project_chd/

Cloning into 'project_chd'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 73 (delta 22), reused 2 (delta 2), pack-reused 29
Receiving objects: 100% (73/73), 2.04 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier


In [12]:
# getting train sets
X_train = pd.read_csv("/content/project_chd/X_train_cleaned.csv")
y_train = pd.read_csv("/content/project_chd/y_train_cleaned.csv")

# getting test sets
X_test = pd.read_csv("/content/project_chd/X_test_cleaned.csv")
y_test = pd.read_csv("/content/project_chd/y_test_cleaned.csv")

In [31]:
# trying linear regression with all variables
stock_reg = LinearRegression().fit(X_train, y_train)
print("R^2 (all variables) = ", stock_reg.score(X_test, y_test))

# trying linear regression with min-max normalizing
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

norm_reg = LinearRegression().fit(X_train_norm, y_train)
print("R^2 (min-max normalized): ", norm_reg.score(X_test_norm, y_test))

# trying linear regression with choosing correlated variables
corr_matrix = X_train.corr()
high_corr_vars = corr_matrix.index[abs(corr_matrix["BMI"]) > 0.5]
X_train_corr = X_train[high_corr_vars]
X_test_corr = X_test[high_corr_vars]

corr_reg = LinearRegression().fit(X_train_corr, y_train)
print("R^2 (correlated variables): ", corr_reg.score(X_test_corr, y_test))

# trying linear regression with polynomial expanders (probably want to test multiple degrees)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

poly_reg = LinearRegression().fit(X_train_poly, y_train)
print("R^2 (polynomial features, degree 2): ", poly_reg.score(X_test_poly, y_test))



R^2 (all variables) =  0.09069686607234828
R^2 (min-max normalized):  0.09069686607234895
R^2 (correlated variables):  0.009601962516968432
R^2 (polynomial features, degree 2):  0.014284325836945277


In [35]:
# Create kNN model - best k seems to be 4
knn = KNeighborsClassifier(n_neighbors=4)

# Fit model
knn.fit(X_train, y_train)

# Make predictions on test set
y_pred = knn.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

R-squared: -0.24068814802691763


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [33]:
# Create decision tree - need to experiment with parameters
dt = DecisionTreeClassifier(max_depth=3, random_state=42)

# Fit model
dt.fit(X_train, y_train)

# Make predictions on test set
y_pred = dt.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

R-squared: -0.14389687406737073
